In [0]:
import gym
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## Policy Iteration
- Policy Evaluation
- Improve Policy

### 初始化环境

In [0]:
environment = gym.make('FrozenLake-v0')

In [3]:
# 打印环境
environment.render()


SFFF
FHFH
FFFH
HFFG


In [4]:
# 查看从一个state到new state的概率
# 从state=6开始(也就是从起点开始), 进行action=1(向下走), 会有可能向左或右走.
# LEFT = 0, DOWN = 1, RIGHT = 2, UP = 3

state = 6
action = 1
environment.P[state][action]

[(0.3333333333333333, 5, 0.0, True),
 (0.3333333333333333, 10, 0.0, False),
 (0.3333333333333333, 7, 0.0, True)]

### Policy Evaluation

首先我们定义用来衡量一个policy好坏的函数.

In [0]:
def policy_evaluation(policy, environment, discount_factor=0.9, theta=1e-9, max_iterations=1000):
    """
    Evaluate a policy given a deterministic environment.

    PARAMETERS
    ----------
    policy: nS*nA的矩阵, 每一个值代表在这个state采取action的概率
    environment: gym提供的环境
    discount_factor: 折扣因子
    theta: 判断是否收敛, 如果value function里面的值改变较小, 则停止迭代
    max_iterations: 最大迭代次数
                    
    RETURNS
    -------
    V: 这个policy对应的value function
    """
    # 记录迭代次数
    evaluation_iterations = 1
    vList = [0] # 记录每次迭代前后v平均值的变化

    # 初始化value function向量, 也就是每个state到游戏结束可能获得的累计reward
    V = np.zeros(environment.nS)

    for i in range(int(max_iterations)):
        # 记录两次迭代value function改变的值, 用来判断是否收敛
        delta = 0
        
        for state in range(environment.nS): # 对所有的状态进行遍历			
            # 记录当前state的value
            v = 0
            
            # 当前state下所有能做的action和做的概率
            for action, action_probability in enumerate(policy[state]):
                # 采取相应的action之后, 下一步的新的state出现的概率
                for state_probability, next_state, reward, terminated in environment.P[state][action]:
                    # 这里修改一下原始reward的值, 每走一步reward减一, 终点reward是10
                    if reward == 0:
                        reward = reward - 1 # 每走一步reward都是-1
                    elif reward == 1:
                        reward = 10
                    # 计算value值
                    v += action_probability * state_probability * (reward + discount_factor * V[next_state])
                
            # 保存一个更新前后最大的差距
            delta = max(delta, abs(V[state] - v))
            
            # 更新state value
            V[state] = v
        
        # 计算前后差值, 用来绘图
        vList.append(np.abs(np.mean(V)))
        
        # 更新迭代次数
        evaluation_iterations += 1

        # 若收敛, 则早停止
        if(delta < theta):
            # print('Policy evaluated in %d iterations' % evaluation_iterations)
            vdeltaList = np.array(vList[1:]) - np.array(vList[:-1])
            return V, vdeltaList
    
    vdeltaList = np.array(vList[1:]) - np.array(vList[:-1])
    return V, vdeltaList

### Policy Iteration

接着我们进行policy的迭代. 

In [10]:
# LEFT = 0, DOWN = 1, RIGHT = 2, UP = 3
environment.P[0][3]

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [0]:
def one_step_lookahead(environment, state, V, discount_factor):
    # Create a vector of dimensionality same as the number of actions
    action_values = np.zeros(environment.nA)

    for action in range(environment.nA):
        # 采取同一个action, 达到不同的state
        for probability, next_state, reward, terminated in environment.P[state][action]:
            if reward == 0:
                reward = -1
            elif reward == 1:
                reward = 10
            action_values[action] += probability * (reward + discount_factor * V[next_state])
    return action_values

In [0]:
def policy_iteration(environment, discount_factor=1.0, max_iterations=1e9):
    # 初始化一个策略, 每一个action都是平均的
    policy = np.ones((environment.nS, environment.nA)) / environment.nA
    print('Step:0;\n V:\n{};\n Policy:\n{}'.format(np.zeros(environment.nS).reshape(4,4), policy))
    # Store the number of policies evaluated
    evaluated_policies = 1
	
    for i in range(int(max_iterations)):	
        # For Early Stopping
        stable_policy = True
        # 首先对当前策略进行评价
        V,_ = policy_evaluation(policy, environment, discount_factor=discount_factor)
        # 检查每一个state
        for state in range(environment.nS):
            # 获得当前state下最大概率的一个action
            current_action = np.argmax(policy[state])
            # 计算这个state出发的action value
            action_values = one_step_lookahead(environment, state, V, discount_factor)
            # 获得最好的动作
            best_action = np.argmax(action_values)
            # 如果动作还在改变, 就说明还在更新
            if(current_action != best_action):
                stable_policy = False
            # 更新policy函数
            policy[state] = np.eye(environment.nA)[best_action]
		
        # Increment the number of policies evaluated
        print('Step:{}, Stable Policy:{};\n V:\n{};\n Policy:\n{}'.format(evaluated_policies, stable_policy, V.reshape(4,4), policy))
        print('='*7)
        evaluated_policies += 1

        # Early stopping
        if(stable_policy):
            print('Evaluated %d policies.' % evaluated_policies)
            return policy, V
    return policy, V

In [21]:
policy, V = policy_iteration(environment=environment, discount_factor=0.9, max_iterations=300)
print(V.reshape(4,4))

Step:0;
 V:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]];
 Policy:
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
Step:1, Stable Policy:False;
 V:
[[-9.95075012 -9.95355297 -9.88926567 -9.95469959]
 [-9.92605845 -9.99999999 -9.7103292  -9.99999999]
 [-9.79456232 -9.3663229  -8.82330857 -9.99999999]
 [-9.99999999 -8.56578645 -5.69360823 -9.99999999]];
 Policy:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
Step:2, Stable Policy:True;
 V:
[[-9.24220004 -9.32443971 -

In [64]:
VE = np.array([-9.95075012, -9.95355297, -9.88926567, -9.95469959, 
        -9.92605845, -9.99999999, -9.7103292,  -9.99999999, 
        -9.79456232, -9.3663229,  -8.82330857, -9.99999999, 
        -9.99999999, -8.56578645, -5.69360823, -9.99999999])
one_step_lookahead(environment, state=14, V=VE, discount_factor=1)

array([-8.69423442, -5.41979822, -5.50563893, -6.46303167])

In [0]:
# LEFT = 0, DOWN = 1, RIGHT = 2, UP = 3
policy

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

### 每一步的选择

In [33]:
print(np.argmax(policy[0]))
environment.P[0][np.argmax(policy[0])]

0


[(0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 4, 0.0, False)]

In [0]:
print(np.argmax(policy[4]))
environment.P[4][np.argmax(policy[4])]

0


[(0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 4, 0.0, False),
 (0.3333333333333333, 8, 0.0, False)]

In [0]:
print(np.argmax(policy[8]))
environment.P[8][np.argmax(policy[8])]

3


[(0.3333333333333333, 9, 0.0, False),
 (0.3333333333333333, 4, 0.0, False),
 (0.3333333333333333, 8, 0.0, False)]

In [0]:
print(np.argmax(policy[9]))
environment.P[9][np.argmax(policy[9])]

1


[(0.3333333333333333, 8, 0.0, False),
 (0.3333333333333333, 13, 0.0, False),
 (0.3333333333333333, 10, 0.0, False)]

In [0]:
print(np.argmax(policy[10]))
environment.P[10][np.argmax(policy[10])]

0


[(0.3333333333333333, 6, 0.0, False),
 (0.3333333333333333, 9, 0.0, False),
 (0.3333333333333333, 14, 0.0, False)]

In [0]:
print(np.argmax(policy[13]))
environment.P[13][np.argmax(policy[13])]

2


[(0.3333333333333333, 13, 0.0, False),
 (0.3333333333333333, 14, 0.0, False),
 (0.3333333333333333, 9, 0.0, False)]